In [ ]:
# GROUP 12
# This notebook designed for Google Colab, may not correct in other environments.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/sample_data/'
!rm -f *
!cp '/content/drive/MyDrive/Colab Notebooks/M2/data.csv' '/content/sample_data'
!chmod 755 -R '/content/sample_data/'

/content/sample_data


In [ ]:
# Install postgresql server
!sudo apt -y install vim bash-completion wget
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ `lsb_release -cs`-pgdg main" | sudo tee  /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-13 postgresql-client-13
!sudo service postgresql start

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
The following additional packages will be installed:
  libgpm2 vim-common vim-runtime xxd
Suggested packages:
  gpm ctags vim-doc vim-scripts
The following NEW packages will be installed:
  bash-completion libgpm2 vim vim-common vim-runtime xxd
0 upgraded, 6 newly installed, 0 to remove and 31 not upgraded.
Need to get 6,890 kB of archives.
After this operation, 33.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 xxd amd64 2:8.0.1453-1ubuntu1.4 [49.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 vim-common all 2:8.0.1453-1ubuntu1.4 [70.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 bash-completion all 1:2.8-1ubuntu1 [168 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgpm2 amd64 1.20.7-5 [15.1 kB]
Get:5 http://archive.ubunt

In [ ]:
# setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
# setup a database with name `m2` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS m2;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE m2;'

ALTER ROLE
NOTICE:  database "m2" does not exist, skipping
DROP DATABASE
CREATE DATABASE


In [ ]:
# creat news table
!sudo -u postgres psql -U postgres -c "CREATE TABLE news(index text,id text,domain TEXT,type TEXT,url TEXT,content TEXT,scraped_at DATE,inserted_at DATE,updated_at DATE,title TEXT,authors TEXT,keywords TEXT,meta_keywords TEXT,meta_description TEXT,tags TEXT,summary TEXT,source TEXT);"

CREATE TABLE


In [ ]:
# load csv
!sudo -u postgres psql -U postgres -c "COPY news FROM '/content/sample_data/data.csv' DELIMITER ',' CSV HEADER ENCODING 'UTF8';"

# connect db
import psycopg2
con = psycopg2.connect(database='postgres', user='postgres',password='postgres',host="127.0.0.1", port="5432")
cur = con.cursor()
# delete null in id
cur.execute('DELETE FROM news WHERE id IS NULL')
# delete text in id
cur.execute("delete FROM news WHERE id ~* '^[a-z]'") 
# set integer for id
cur.execute('ALTER TABLE news ALTER COLUMN id TYPE integer USING (id::integer)') 
con.commit()
con.close

COPY 1000000


In [ ]:
con = psycopg2.connect(database='postgres', user='postgres',password='postgres',host="127.0.0.1", port="5432")
cur = con.cursor()
# does one domain only has one type?
cur.execute('select distinct domain, count(distinct type) from news group by domain having count( distinct type)>1 order by domain')
if cur.fetchall()==[]:
  print("One domain only has one type.")
con.close

One domain only has one type.


<function connection.close>

In [ ]:
con = psycopg2.connect(database='postgres', user='postgres',password='postgres',host="127.0.0.1", port="5432")
cur = con.cursor()
# does 'authors' has NULL?
cur.execute('select count(id) from news where authors IS NULL')
null_rows = cur.fetchall()
if null_rows!=[]:
  print(null_rows[0][0],"rows in 'authors' are null.")
con.close

457564 rows in 'authors' are null.


<function connection.close>

In [ ]:
con = psycopg2.connect(database='postgres', user='postgres',password='postgres',host="127.0.0.1", port="5432")
cur = con.cursor()
# Are 'scraped_at', 'inserted_at' and 'updated_at' identical?
cur.execute('SELECT DISTINCT scraped_at, inserted_at, updated_at FROM news')
if len(cur.fetchall())==1:
  print("'scraped_at', 'inserted_at' and 'updated_at' are identical")
con.close

'scraped_at', 'inserted_at' and 'updated_at' are identical


<function connection.close>

In [ ]:
# creat table domain
!sudo -u postgres psql -U postgres -c "DROP table IF EXISTS domain;"
!sudo -u postgres psql -U postgres -c "SELECT id,domain,type into domain from news;"
con = psycopg2.connect(database='postgres', user='postgres',password='postgres',host="127.0.0.1", port="5432")
cur = con.cursor()
# delete dup domains
cur.execute('explain analyse delete from domain d where d.id not in (select min(id) from domain group by domain)')
# delete id
cur.execute('ALTER TABLE domain DROP COLUMN id')
# set domain as PK
cur.execute('ALTER TABLE domain ADD PRIMARY KEY (domain)')
con.commit()

DROP TABLE
SELECT 999934
